## Setup Imports

(not relevant to repo, can skip) 

Adding module path to run correctly in examples folder

In [2]:
import sys
import os
import numpy as np

# Add the path to the folder containing the module
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src


In [ ]:
from common.plotting import plot_gif,plot_img
import wandb
import math

api = wandb.Api()

run = api.run("loc_gsopt/teleport_select_100_data_downlink_7_5=CAPELLA=3000000/rnloat3a") #"<entity>/<project>/<run_id>")
# plot_img(run.summary.gs_list,f"gs_all.png")
# plot_img(run.summary.lat_long,f"images/gs_all.png")


# runs = api.runs("loc_gsopt/free_nelder_data_downlink_5_6=CAPELLA=500000000")
# for index, run in enumerate(runs):
#     plot_img(run.summary.gs_list,f"gs_all_{index}.png")

[[49.489581, 40.466124], [-3.87687500000004, 43.449398], [-58.7191239, -34.4107537], [-77.759132, 39.599098], [-122.28063, 38.244876], [138.619292, -34.81735], [140.373055555556, 36.5316666666667]]


<Figure size 640x480 with 1 Axes>

In [ ]:
from common.utils import load_earth_data,mp_compute_contact_times,contactExclusion # compute_earth_interior_angle
from common.sat_gen import satellites_from_constellation
import brahe as bh

# Make sure to load in earth inertial data every start time!
load_earth_data('data/iau2000A_finals_ab.txt',False)

satellites = satellites_from_constellation("CAPELLA", False)

epc_start = bh.Epoch(2025, 
                         4, 
                         1, 
                         17, 
                         23, 
                         40.69) 
    
epc_end = bh.Epoch(2025, 
                         4, 
                         8, 
                         17, 
                         23, 
                         40.69)  
    

contacts, _ = mp_compute_contact_times(satellites, run.summary.lat_long ,epc_start, epc_end, False)
_, contacts_exclusion_secs = contactExclusion(contacts,cfg)
run.summary["gs_list"] = gs_list_plot 
run.summary["contact_num"] = len(contacts_exclusion_secs) 
run.summary["seconds"] = np.sum(contacts_exclusion_secs)
run.summary["data_downlink"] = np.sum(contacts_exclusion_secs)*cfg.scenario.datarate

